# STAGE C QUIZ FOR HAMOYE

Importing the Relevant Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

from sklearn.linear_model import LogisticRegression

Loading the Required Data

In [2]:
Quiz = pd.read_csv(r"C:\Users\RPN\Desktop\Quiz.csv")
Quiz.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


Working with Train-Test-Split

Droping the irrelevant variable

Since stab is not needed, it will be dropped leaving stabf as the only dependent variable

In [3]:
from sklearn.model_selection import train_test_split
x = Quiz.drop(columns=['stab', 'stabf'])
y = Quiz['stabf']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

Standardize the Dataset

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
normalised_train_df = scaler.fit_transform(X_train)
normalised_train_df = pd.DataFrame(normalised_train_df, columns = X_train.columns)
normalised_test_df = scaler.transform(X_test)
normalised_test_df = pd.DataFrame(normalised_test_df, columns = X_test.columns)

# WORKING ON THE QUIZ QUESTIONS

# Question 20

In [5]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(random_state=1)
etc.fit(normalised_train_df, y_train)
predictions_etc = etc.predict(normalised_test_df)

C:\Users\RPN\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [6]:
featimp_etc = zip(etc.feature_importances_, x.columns)
sorted(featimp_etc, reverse=True)

[(0.11917257167894016, 'tau1'),
 (0.11846716732664228, 'tau2'),
 (0.11498867698767354, 'tau4'),
 (0.1087742681346329, 'tau3'),
 (0.09713771389759826, 'g3'),
 (0.09533512433774224, 'g2'),
 (0.09434916387161776, 'g4'),
 (0.08865189998921906, 'g1'),
 (0.04172889918418444, 'p4'),
 (0.04128050894817886, 'p3'),
 (0.041211994463249316, 'p1'),
 (0.03890201118032111, 'p2')]

The feature with the most importance and least importance are TAU2 and P1

# Question 17

In [7]:
from sklearn.model_selection import RandomizedSearchCV
param_distributions = {'n_estimators': [50, 100, 300, 500, 1000], 'min_samples_split': [2,3,5,7,9], 
                       'min_samples_leaf': [1,2,4,6,8], 'max_features':['auto', 'sqrt', 'log2', None],'random_state': [1]}
randomcv = RandomizedSearchCV(ExtraTreesClassifier(), param_distributions, n_iter=10, scoring='accuracy', n_jobs=-1, refit=True,
                              cv=5, verbose=1, random_state=1)
randomcv.fit(normalised_train_df, y_train)
predictions_randomcv = randomcv.predict(normalised_test_df)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.5min finished


In [8]:
randomcv.best_params_

{'random_state': 1,
 'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 8,
 'max_features': None}

# Question 16

Building a light gradient boosting model (LGBM)

Accuracy (LGBM)

In [9]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state=1)
lgbm.fit(normalised_train_df, y_train)
predictions_lgbm = lgbm.predict(normalised_test_df)

In [11]:
from sklearn import metrics
accuracy_lgbm = metrics.accuracy_score(y_test, predictions_lgbm)
print('Accuracy: {}'.format(round(accuracy_lgbm, 2)))

Accuracy: 0.94


According to the options given i choose 0.9375

# Question 15

Building an extreme gradient boosting model classifier (XGBC)

Accuracy (XGBC)

In [12]:
from xgboost import XGBClassifier
xgbc = XGBClassifier(max_depth=3, learning_rate=0.1, random_state=1)
xgbc.fit(normalised_train_df, y_train)
predictions_xgbc = xgbc.predict(normalised_test_df)

In [13]:
accuracy_xgbc = metrics.accuracy_score(y_test, predictions_xgbc)
print('Accuracy: {}'.format(round(accuracy_xgbc, 2)))

Accuracy: 0.92


According to the options given i choose 0.9195

# QUESTION 14

Building a random forest classifier model (RFC)

Accuracy (RFC)

In [14]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=1)
rfc.fit(normalised_train_df, y_train)
predictions_rfc = rfc.predict(normalised_test_df)

C:\Users\RPN\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [15]:
accuracy_rfc = metrics.accuracy_score(y_test, predictions_rfc)

print('Accuracy: {}'.format(round(accuracy_rfc, 2)))

Accuracy: 0.89


According to the options given i choose 0.9295 because my answer wasnt there

According to an online source. My code for the random forest classifier model is 0.89 because of an outdated version of sklearn.
but through group work and assistance from fellow colleages i got 0.9295 4.d.p

# THANK YOU!